In [1]:
import pandas as pd
import json
import string

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [20]:
columns = [
    'season_id', 'queue_id', 'game_id', 'game_version', 'platform_id',
    'game_mode', 'map_id', 'game_type',
    'champion_1', 'champion_2', 'champion_3',
    'champion_4', 'champion_5', 'champion_6',
    'champion_7', 'champion_8', 'champion_9', 'champion_10',
    'game_duration', 'game_creation',
    'team_1_outcome', 'team_2_outcome']

with open('champions.json', 'r') as f:
    champions = json.load(f)

df = pd.read_csv('match_data.txt', names=columns, low_memory=False, encoding='utf-8')

In [22]:
# map champion id to champion name
for i in range(1,11):
    champion_n = 'champion_{}'.format(i)
    df[champion_n] = df[champion_n].apply(lambda x: champions[str(x)])

In [26]:
df = df[(df.game_mode == ' CLASSIC') | (df.game_mode == ' ARAM')]

In [28]:
df.to_csv('match_data_champions.csv', index=False)

In [ ]:
# produce dataset with winning teams
# these are the ones used for the recommendations
# I'll admit it's mega ugly and hacky
list_of_winning_teams = []
for index, row in df.iterrows():
    d = {}
    start = 1
    
    # add to the dict all the columns
    for i in row.keys():
        d[i] = row[i]
    # set all the values to 0
    for c in champions.values():
        d[c] = 0
    # if team 1 wins, we collect the first 5 champions
    # e.g. champion_1, champion_2, ...,champion_5
    # otherwise, we use the last 5
    # e.g. champion_6, champion_7, ...,champion_10
    if d['team_1_outcome'] == 0:
        start = 6
    for e in range(start,start+5):
        d[row['champion_{}'.format(e)]] = 1
    list_of_winning_teams.append(d)

In [34]:
winning_teams_final = pd.DataFrame(list_of_winning_teams)
# for this dataset I only care about the teams
winning_teams_final.drop(columns=columns, inplace=True)

In [38]:
winning_teams_final.to_csv('winning_teams.csv', index=False, header=True)